Steal from https://github.com/hecko/pySigfox/blob/master/pySigfox/pySigfox.py

In [1]:
import os
import sys
import json
import requests
from pprint import pprint

def load_api_secrets(filename):
    """Load credentials from json file."""
    try:
        with open(filename, 'r') as fp:
            api_params = json.load(fp)
    except Exception as e:
        print('Failed to load API secrets key: {}'.format(e))
        api_params = None
    return api_params

In [2]:
api_secrets_file = '/Users/robincole/Desktop/SigFox_API_key.json'

In [3]:
api_params = load_api_secrets(api_secrets_file)

API_login = api_params['API_login']
API_pass = api_params['API_pass']

In [4]:
class Sigfox:
    def __init__(self, login, password, debug=False):
        if not login or not password:
            raise Exception("Please define login and password when initiating pySigfox class!")
        self.login    = login
        self.password = password
        self.api_url  = 'https://backend.sigfox.com/api/'
        self.debug = debug

    def login_test(self):
        """Try to login into the  Sigfox backend API - if unauthorized or any other issue raise Exception
        """
        url = self.api_url + 'devicetypes'
        r = requests.get(url, auth=requests.auth.HTTPBasicAuth(self.login, self.password))
        if r.status_code != 200:
            raise Exception("Unable to login to Sigfox API: " + str(r.status_code))

    def device_types_list(self):
        """Return list of device types dictionaries
        """
        out = []
        url = self.api_url + 'devicetypes'
        r = requests.get(url, auth=requests.auth.HTTPBasicAuth(self.login, self.password))
        for device in json.loads(r.text)['data']:
            out.append(device)
        return out

    def device_info(self, device_id):
        """Return information about specific device
        """
        out = []
        url = self.api_url + 'devices/' + str(device_id)
        if self.debug:
            print("Connecting to " + url)
        r = requests.get(url, auth=requests.auth.HTTPBasicAuth(self.login, self.password))
        try:
            return json.loads(r.text)
        except Exception as e:
            pprint(r.text)
            raise

    def device_list(self, device_type):
        """Return array of dictionaries - one array item per device.
        :param device_type: Return only devices of a certain type.
            This is a object from device_groups_list()
        :return: List of dictionaries 
        :rtype: list
        """
        device_type_ids = []
        out = []
        url = self.api_url + 'devicetypes/' + device_type['id'] + '/devices'
        r = requests.get(url, auth=requests.auth.HTTPBasicAuth(self.login, self.password))
        try:
            out.extend(json.loads(r.text)['data'])
        except Exception as e:
            print("Unable to access data from returned RESP API call: " + str(e))
            pprint(r.text)
            raise
        return out

    def device_messages(self, device, limit=10):
        """Return array of 10 last messages from specific device.
           
        :param device: Device object
        :param limit: how many messages to retrieve - max limit 100
        :type limit: int
        """

        url = self.api_url + 'devices/' + str(device['id']) + '/messages?limit=' + str(limit)
        r = requests.get(url, auth=requests.auth.HTTPBasicAuth(self.login, self.password))

        try:
            out = json.loads(r.text)['data']
        except Exception as e:
            pprint(r.text)
            raise

        return out

    def device_messages_page(self, url):
        """Return array of message from paging URL.
        """
        out = []
        r = requests.get(url, auth=requests.auth.HTTPBasicAuth(self.login, self.password))
        out.extend(json.loads(r.text)['data'])
        try:
            json.loads(r.text)['paging']['next']
            out.extend(self.device_messages_page(json.loads(r.text)['paging']['next']))
        except Exception as e:
            # no more pages
            pass

        return out

Run the example https://github.com/hecko/pySigfox/blob/master/example.py

In [5]:
s = Sigfox(login=API_login, password=API_pass)

In [6]:
s.login_test()

In [7]:
s.device_types_list()

[{'contract': '5a706c203c878968fedcd0ff',
  'description': 'Auto created device type for EVK user : Robin Cole',
  'group': '5aa59d3d5005741b7de9aa50',
  'id': '5aa59d3d5005741b7de9aa51',
  'keepAlive': 0,
  'name': 'Pycom - lopy',
  'payloadType': 'String'}]

In [8]:
print("Getting list of all devices:")
for device_type in s.device_types_list():
    for device in s.device_list(device_type):
        pprint(device)
        last_device = device

Getting list of all devices:
{'activationTime': 1520803295634,
 'automaticRenewal': False,
 'averageRssi': -106.24595,
 'averageSignal': 45.19108,
 'averageSnr': 45.19108,
 'contractId': '5a706c203c878968fedcd0ff',
 'id': '4D30A7',
 'last': 1520803477,
 'lat': 0.0,
 'lng': 0.0,
 'name': 'Device 004d30a7',
 'preventRenewal': True,
 'state': 0,
 'tokenEnd': 1552339295634,
 'tokenType': 'CONTRACT',
 'type': '5aa59d3d5005741b7de9aa51'}


In [9]:
print("== Last 3 messages from " + last_device['name'] + ":")
pprint(s.device_messages(last_device, limit=3))

== Last 3 messages from Device 004d30a7:
[{'country': 'GBR',
  'data': '48656c6c6f20576f726c64',
  'device': '4D30A7',
  'groupId': '5aa59d3d5005741b7de9aa50',
  'linkQuality': 'GOOD',
  'nbFrames': 3,
  'operator': 'SIGFOX_UK_Arqiva',
  'rinfos': [{'delay': 1.0449999570846558,
              'lat': '51.0',
              'lng': '0.0',
              'tap': '0B79'},
             {'delay': 2.437000036239624,
              'lat': '51.0',
              'lng': '0.0',
              'tap': '1DB6'}],
  'seqNumber': 1,
  'snr': '46.48',
  'time': 1520803477},
 {'country': 'GBR',
  'data': '0102030405060708090a0b0c',
  'device': '4D30A7',
  'groupId': '5aa59d3d5005741b7de9aa50',
  'linkQuality': 'GOOD',
  'nbFrames': 3,
  'operator': 'SIGFOX_UK_Arqiva',
  'rinfos': [{'delay': 0.574999988079071,
              'lat': '51.0',
              'lng': '0.0',
              'tap': '0B79'}],
  'seqNumber': 0,
  'snr': '43.35',
  'time': 1520803295}]


In [10]:
last_device

{'activationTime': 1520803295634,
 'automaticRenewal': False,
 'averageRssi': -106.24595,
 'averageSignal': 45.19108,
 'averageSnr': 45.19108,
 'contractId': '5a706c203c878968fedcd0ff',
 'id': '4D30A7',
 'last': 1520803477,
 'lat': 0.0,
 'lng': 0.0,
 'name': 'Device 004d30a7',
 'preventRenewal': True,
 'state': 0,
 'tokenEnd': 1552339295634,
 'tokenType': 'CONTRACT',
 'type': '5aa59d3d5005741b7de9aa51'}

In [18]:
last_device['name']

'Device 004d30a7'

Get the last message only

In [11]:
data = s.device_messages(last_device, limit=1)
data

[{'country': 'GBR',
  'data': '48656c6c6f20576f726c64',
  'device': '4D30A7',
  'groupId': '5aa59d3d5005741b7de9aa50',
  'linkQuality': 'GOOD',
  'nbFrames': 3,
  'operator': 'SIGFOX_UK_Arqiva',
  'rinfos': [{'delay': 1.0449999570846558,
    'lat': '51.0',
    'lng': '0.0',
    'tap': '0B79'},
   {'delay': 2.437000036239624, 'lat': '51.0', 'lng': '0.0', 'tap': '1DB6'}],
  'seqNumber': 1,
  'snr': '46.48',
  'time': 1520803477}]

In [12]:
print(bytes.fromhex(data[0]['data']).decode('utf-8')) # print the message decoded

Hello World


In [15]:
lat = data[0]['rinfos'][0]['lat']
lat

'51.0'

In [17]:
lng = data[0]['rinfos'][0]['lng']
lng

'0.0'